In [1]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer

In [2]:
def clean(data):
    data=data.drop(['rewards_burned','convenience_fee_bookings'],axis=1)
    data['city']=data['city'].replace('','Unknown')
    data['coupon_value']=data['coupon_value'].fillna(0)
    data=data.dropna()
    return data

In [3]:
def transform(data):
    popular_destinations=['IXL', 'KUU', 'DHM', 'PYG', 'SLV', 'AGR', 'JAI', 'UDR', 'SXR', 'ATQ', 'COK', 'HBX', 'TIR', 'GOI', 'GOI', 'PNQ', 'UDR', 'IXZ']
    metro_cities=['BOM', 'DEL', 'BLR', 'HYD', 'MAA', 'CCU', 'PNQ', 'AMD', 'STV', 'VTZ']
    rich_cities=["Goa","Sikkim","Delhi","Chandigarh","Haryana","Mumbai", "Delhi", "Kolkata", "Bangalore", "Hyderabad", "Chennai", "Ahmedabad", "Pune", "Surat", "Visakhapatnam", "Jaipur", "Lucknow", "Nagpur", "Indore", "Patna"]
    df=data.copy()
    df['isMetroCity'] = df.apply(lambda row: 1 if row['arrival_station'] in metro_cities or row['departure_station'] in metro_cities else 0,axis=1) 
    df['isPopularDestination'] = df.apply(lambda row: 1 if row['arrival_station'] in popular_destinations or row['departure_station'] in popular_destinations else 0,axis=1)
    df['isRichRegion'] = df.apply(lambda row: 1 if row['region'] in rich_cities or row['city'] in rich_cities or row['country']!='India' else 0,axis=1)
    df['arrival_date'] = None
    df['departure_date'] = None
    df['arrival_time']=None
    df['departure_time']=None
    for i, row in df.iterrows():
        if row['platform1'] == 'APP':
            arrival_time = row['arrival_datetime'][2:]
        else:
            arrival_time = row['arrival_datetime']
        arrival_date = pd.to_datetime(arrival_time, format='%Y-%m-%dT%H:%M:%S.%fZ').date()
        df.at[i, 'arrival_date'] = arrival_date
    for i, row in df.iterrows():
        if row['platform1'] == 'APP':
            arrival_time = row['arrival_datetime'][2:]
        else:
            arrival_time = row['arrival_datetime']
        arrival_date = pd.to_datetime(arrival_time, format='%Y-%m-%dT%H:%M:%S.%fZ').time()
        df.at[i, 'arrival_time'] = arrival_date
    for i, row in df.iterrows():
        if row['platform1'] == 'APP':
            departure_time = row['departure_datetime'][2:]
        else:
            departure_time = row['departure_datetime']
        departure_date = pd.to_datetime(departure_time, format='%Y-%m-%dT%H:%M:%S.%fZ').date()
        df.at[i, 'departure_date'] = departure_date
    for i, row in df.iterrows():
        if row['platform1'] == 'APP':
            arrival_time = row['departure_datetime'][2:]
        else:
            arrival_time = row['departure_datetime']
        arrival_date = pd.to_datetime(arrival_time, format='%Y-%m-%dT%H:%M:%S.%fZ').time()
        df.at[i, 'departure_time'] = arrival_date
    df=df.rename(columns={'event_time':'event_datetime'})
    df['event_date'] = pd.to_datetime(df['event_datetime']).dt.date
    df['event_time'] = pd.to_datetime(df['event_datetime']).dt.time
    df=df.drop(['event_datetime','arrival_datetime','departure_datetime'],axis=1)
    return df

In [4]:
def merge_GA_Synapse(df):
    GA_data=df
    Synapse_data=pd.read_csv("Synapse_Results.csv")
    result=pd.merge(GA_data,Synapse_data,left_on='user_id',right_on='loyaltyId')
    result=result.drop('loyaltyId',axis=1)
    result=result.drop_duplicates()
    return result

In [5]:
def get_urgency(df):
    df['departure_date']=pd.to_datetime(df['departure_date'])
    df['event_date']=pd.to_datetime(df['event_date'])
    df['urgency_days'] = (df['departure_date'] - df['event_date']).dt.days
    return df

In [6]:
def get_visitations(df):
    df.sort_values(by=["user_id", "event_date", "event_time"], inplace=True)
    df['visitations']=None
    user_dict={}
    for i,row in df.iterrows():
        if (row['user_id'],row['departure_date'],row['departure_station'],row['arrival_station']) not in user_dict:
            user_dict[(row['user_id'],row['departure_date'],row['departure_station'],row['arrival_station'])]=1
            df.at[i,'visitations']=user_dict[(row['user_id'],row['departure_date'],row['departure_station'],row['arrival_station'])]
        elif (row['user_id'],row['departure_date'],row['departure_station'],row['arrival_station']) in user_dict and row['isbooked']==0:
            user_dict[(row['user_id'],row['departure_date'],row['departure_station'],row['arrival_station'])]+=1
            df.at[i,'visitations']=user_dict[(row['user_id'],row['departure_date'],row['departure_station'],row['arrival_station'])]
        elif (row['user_id'],row['departure_date'],row['departure_station'],row['arrival_station']) in user_dict and row['isbooked']==1:
            user_dict[(row['user_id'],row['departure_date'],row['departure_station'],row['arrival_station'])]+=1
            df.at[i,'visitations']=user_dict[(row['user_id'],row['departure_date'],row['departure_station'],row['arrival_station'])]
            user_dict[(row['user_id'],row['departure_date'],row['departure_station'],row['arrival_station'])]=0
    return df

In [7]:
def DataPipeline(df):
    df=clean(df)
    df=transform(df)
    df=merge_GA_Synapse(df)
    df=get_urgency(df)
    df=get_visitations(df)
    return df